In [ ]:
########## Monter mon drive sur colab
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#Importation des packages
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
import nltk
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
nltk.download('vader_lexicon')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
!pip install spacy
!python -m spacy download fr_core_news_md
import spacy
import fr_core_news_md
!pip install -U textblob
!pip install -U textblob-fr

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
     |████████████████████████████████| 88.6MB 1.2MB/s 
  Created wheel for fr-core-news-md: filename=fr_core_news_md-2.2.5-cp37-none-any.whl size=90338490 sha256=c6c6d82839eecb5a661ae1709fcb97ce0c17b69b1f19754342fd23068d576335
  Stored in directory: /tmp/pip-ephem-wheel-cache-s8qdk5lz/wheels/c6/18/b6/f628642acc7872a53cf81269dd1c394d96da69564ccfac5425
Successfully built fr-core-news-md
✔ Download and installation successful
You can now load the model via spacy.load('fr_core_news_md')
Requirement already up-to-date: textblob in /usr/local/lib/python3.7/dist-packages (0.15.3)
     |█████████████████████

In [ ]:
#l'URL du page index
master="http://www.allocine.fr"
pages_s=[]
#Recupere les ahref de toutes les films d'un genre spécifié
for i in range(10):
   index="http://www.allocine.fr/series-tv/genre-13030/?page="+str(i+1)
   p=requests.get(index)
   script= html.fromstring(p.content)
   pages_s.extend(script.xpath('//a[@class="meta-title-link"]/@href'))


titres_s=[]
avis_s=[]
rate_s=[]

#boucler sur tous les films et recuperer le titre les avis et les scores de chaque film
for i in pages_s:
  index=master+i
  p=requests.get(index)
  script= html.fromstring(p.content)
  titres =(script.xpath('//div[@class="titlebar-title titlebar-title-lg"]//text()')[0])
  avis = script.xpath('//div[@class="content-txt review-card-content"]/text()')
  rate = script.xpath('//div[@class="stareval stareval-medium stareval-theme-default"]/span[@class="stareval-note"]/text()')
  for j,k in zip(avis,rate):
    titres_s.append(titres)
    avis_s.append(j)
    rate_s.append(k)

#Sauvegarder le resultat de Scrapping dans une dataframe
d={"Titre":titres_s,"Avis":avis_s,"Score":rate_s}
df=pd.DataFrame(d)
df.to_csv('/content/drive/My Drive/TM/Avis_Medicale.csv' , encoding = "UTF-8",sep =";" , index = False)

In [ ]:
#Enlever les Stop Word
def remove_stop_words(corpus):
    french_stop_words = stopwords.words('french')
    word_tokens = word_tokenize(corpus) 
    filtered_sentence = [] 
    for w in word_tokens: 
        if w not in french_stop_words: 
            filtered_sentence.append(w) 
    return ' '.join(filtered_sentence)

In [ ]:
#Stemming
"""
def get_stemmed_text(sentence):
    token_words=word_tokenize(sentence)
    stem_sentence=[]
    porter=PorterStemmer()
    for word in token_words:
        stem_sentence.append(porter.stem(word))
    return ' '.join(stem_sentence)
"""

"\ndef get_stemmed_text(sentence):\n    token_words=word_tokenize(sentence)\n    stem_sentence=[]\n    porter=PorterStemmer()\n    for word in token_words:\n        stem_sentence.append(porter.stem(word))\n    return ' '.join(stem_sentence)\n"

In [ ]:
#Lemmatization (n'est pas fiable)
"""
def get_lemmatized_text(corpus):
    wordnet_lemmatizer = WordNetLemmatizer()    
    token_words=word_tokenize(corpus)
    lem_sentence=[]
    for w in token_words:
        lem_sentence.append(wordnet_lemmatizer.lemmatize(w))
    return ' '.join(lem_sentence)
"""

"\ndef get_lemmatized_text(corpus):\n    wordnet_lemmatizer = WordNetLemmatizer()    \n    token_words=word_tokenize(corpus)\n    lem_sentence=[]\n    for w in token_words:\n        lem_sentence.append(wordnet_lemmatizer.lemmatize(w))\n    return ' '.join(lem_sentence)\n"

In [ ]:
#Lematization avec fr_core_news_md fiable que WordNetLemmatizer
nlp = fr_core_news_md.load()
def get_lemmatized_text(corpus):
    filtered_sentence = []
    t= nlp(u""+corpus)
    for i in t:
       filtered_sentence.append(i.lemma_)
    return ' '.join(filtered_sentence)


In [ ]:
#enlever les ponctuation et les nombres
def get_unpuncuated_text(corpus):
    words = nltk.word_tokenize(corpus)
    words=[word.lower() for word in words if word.isalpha()]
    return ' '.join(words)

In [ ]:
#Preprocessing complet
def preprocessing(corpus):
  corpus= remove_stop_words(corpus)
  corpus=get_lemmatized_text(corpus)
  corpus =get_unpuncuated_text(corpus)
  return corpus


In [ ]:
# sample text for performing tokenization
text = "In Brazil they drive on the right-hand side of the road. Brazil has a large coastline on the eastern side of South America"
# importing word_tokenize from nltk
from nltk.tokenize import word_tokenize
# Passing the string text into word tokenize for breaking the sentences
token = word_tokenize(text)
token

['In',
 'Brazil',
 'they',
 'drive',
 'on',
 'the',
 'right-hand',
 'side',
 'of',
 'the',
 'road',
 '.',
 'Brazil',
 'has',
 'a',
 'large',
 'coastline',
 'on',
 'the',
 'eastern',
 'side',
 'of',
 'South',
 'America']

In [ ]:
#Preprocessing sur les description
desc_clean=[]
for i in desc:
  desc_clean.append(preprocessing(i))

In [ ]:
#Preprocessing sur les avis
def clean_Avis(a):
   c=[]
   for i in a:
      x=i.replace("\n","")
      for j in range(20):
          if "  " in x:
               x=x.replace("  ","")
      c.append(x)
   for k in range(0,len(c)):
        if c[k] == "":
          c[k]=None
          
   return c 
clean_avis=clean_Avis(avis)


Recherche par titre

In [ ]:
#Grouper les avis (une fct qui retourne les avis d'un film donné ordonné par score)
def grouper_par_score(t):
  d={"Titre":titreR,"Avis":clean_avis,"Score":rating}
  a= pd.DataFrame(d) 
  a= a[a["Titre"] == t]
  a=a.dropna(axis=0)
  a=a.sort_values(by=['Score'], ascending=False)
  return a


In [ ]:
#Recherche par titre (une fct qui retourne toute les infos d'un film donné)
def recherche_par_titre(t):
  for i in titre:
    if i == t:
      print("Titre :"+t)
      print("Genre :"+genre[titre.index(i)])
      print("Description :"+desc[titre.index(i)])
      print("Avis :")
      print(grouper_par_score(t))
recherche_par_titre("Titans")

Titre :Titans
Genre :Action
Description :["Dick Grayson, alias Robin,\xa0sort de l’ombre pour devenir le chef de file d’un groupe de nouveaux héros constitué notamment de Starfire, Raven ou encore Hawk et Dove. Peu à peu, il\xa0s'affranchit de Batman pour devenir Nightwing...."]
Avis :
    Titre                                               Avis Score
1  Titans  Bon début sur cette série mais flop total sur ...   4,0
0  Titans  Cette série est une véritable déception ! A pa...   3,2
2  Titans  Vu les deux premiers épisodes, il est certes p...   1,5
5  Titans  je n'ai pas accroché à cette série car ce n'es...   0,5


In [ ]:
#Analyser les sentiments des avis
def analyser_avis():
  from textblob import TextBlob
  from textblob import Blobber
  from textblob_fr import PatternTagger, PatternAnalyzer
  tb = Blobber(pos_tagger=PatternTagger(), analyzer=PatternAnalyzer())
  res=[]
  for i in avis:
      text = tb(u""+i)
      res.append(text.sentiment[0])
  ress=[]
  for i in range(0,len(res)):
      res[i]=round(res[i],2)
      if res[i] > 0:
        ress.append(str("Positive de "+ str(int(res[i]*100))+ "%"))
      elif res[i] == 0:
        ress.append(str("Neutre"))
      else :
        ress.append(str("Négative de "+ str(int(res[i]*100*-1))+ "%"))
  return [res,ress]

#Grouper les avis (une fct qui retourne les avis d'un film donné ordonné par analyse de sens) 
def grouper_par_avis(t):
  av=analyser_avis()
  d={"Titre":titreR , "Avis":clean_avis , "Sentiment": av[1] , "ordre": av[0]}
  a= pd.DataFrame(d) 
  a= a[a["Titre"] == t]
  a=a.dropna(axis=0)
  a=a.sort_values(by=["ordre"], ascending=False)
  a=a.drop(["ordre"], axis='columns')
  return a

#Recherche par titre (une fct qui retourne toute les infos d'un film donné)
def recherche_par_titre_AS(t):
  for i in titre:
    if i == t:
      print("Titre :"+t)
      print("Genre :"+genre[titre.index(i)])
      print("Description :"+desc[titre.index(i)])
      print("Avis :")
      #Vous trouverez ici la fonction grouper et analyser par avis
      print(grouper_par_avis(t))

In [ ]:
recherche_par_titre_AS("Titans")

Titre :Titans
Genre :Action
Description :["Dick Grayson, alias Robin,\xa0sort de l’ombre pour devenir le chef de file d’un groupe de nouveaux héros constitué notamment de Starfire, Raven ou encore Hawk et Dove. Peu à peu, il\xa0s'affranchit de Batman pour devenir Nightwing...."]
Avis :
    Titre                                               Avis        Sentiment
1  Titans  Bon début sur cette série mais flop total sur ...  Positive de 21%
2  Titans  Vu les deux premiers épisodes, il est certes p...   Négative de 2%
0  Titans  Cette série est une véritable déception ! A pa...   Négative de 6%
5  Titans  je n'ai pas accroché à cette série car ce n'es...  Négative de 15%


Recherche par description  en utilisant cosine

In [ ]:
#L'indice de similarité cosine (avec le resultat de TfIdf) 
#une fct qui retourne toute les infos des dix films similaires à une description donnée
def tf_idf(query, description):
    query=preprocessing(query)
    desc_clean=[]
    for i in description:
       desc_clean.append(preprocessing(i))
    import numpy as np
    tfidf_vectorizer = TfidfVectorizer()
    tfidf_desc = tfidf_vectorizer.fit_transform(desc_clean)
    q = tfidf_vectorizer.transform([query])
    cs = cosine_similarity(q, tfidf_desc)
    res= cs[0]
    result_list = [] #index
    sim = [] #similarité
    nb = 10
    while nb > 0:
        index = np.argmax(res)
        result_list.append(index)
        sim.append(res[index])
        res[index] = 0
        nb =nb - 1

    print("les 10 films similaires à votre query est:")
    for i,j in zip(result_list,sim):
            print("Titre :"+titre[i])
            s=int(j*100)
            print("score de similarité :"+str(s)+"%")
            print("Genre :"+genre[i])
            print("Description :"+desc[i])

In [ ]:
tf_idf(desc[0],desc)

les 10 films similaires à votre query est:
Titre :Titans
score de similarité :100%
Genre :Action
Description :["Dick Grayson, alias Robin,\xa0sort de l’ombre pour devenir le chef de file d’un groupe de nouveaux héros constitué notamment de Starfire, Raven ou encore Hawk et Dove. Peu à peu, il\xa0s'affranchit de Batman pour devenir Nightwing...."]
Titre :Robin des bois
score de similarité :10%
Genre :Aventure
Description :["Après 5 ans de croisades, Robin de Locksley rentre enfin chez lui pour découvrir avec effroi que les tyrannique Shérif de Nothingham et Guy de Gisbourne y règnent en maîtres absolus. Pour la population, tout n'est que désolation et misère...", ' Afin de rétablir un ordre plus juste, Robin et ses compagnons Will Scarlett, Allan A Dale et Petit Jean investissent la forêt de Sherwood et deviennent des hors-la-loi... La légende de Robin des Bois est née.']
Titre :Dick le rebelle
score de similarité :10%
Genre :Aventure
Description :['Au 18ème siècle, les aventures de Dic

In [ ]:
#L'indice de similarité cosine (avec le resultat de TF) 
#une fct qui retourne toute les infos des dix films similaires à une description donnée
def tf(query, description):
    import numpy as np
    query=preprocessing(query)
    desc_clean=[]
    for i in description:
       desc_clean.append(preprocessing(i))
    tf_vectorizer = CountVectorizer(binary=False, ngram_range=(1, 2))
    tf_desc = tf_vectorizer.fit_transform(desc_clean)
    q = tf_vectorizer.transform([query])
    cs = cosine_similarity(q, tf_desc)
    res= cs[0]
    result_list = [] #index
    sim = [] #similarité
    nb = 10
    while nb > 0:
        index = np.argmax(res)
        result_list.append(index)
        sim.append(res[index])
        res[index] = 0
        nb =nb - 1

    print("les 10 films similaires à votre query est:")
    for i,j in zip(result_list,sim):
            print("Titre :"+titre[i])
            s=int(j*100)
            print("score de similarité :"+str(s)+"%")
            print("Genre :"+genre[i])
            print("Description :"+desc[i])


In [ ]:
tf(desc[0],desc)

les 10 films similaires à votre query est:
Titre :Titans
score de similarité :100%
Genre :Action
Description :["Dick Grayson, alias Robin,\xa0sort de l’ombre pour devenir le chef de file d’un groupe de nouveaux héros constitué notamment de Starfire, Raven ou encore Hawk et Dove. Peu à peu, il\xa0s'affranchit de Batman pour devenir Nightwing...."]
Titre :Rome
score de similarité :11%
Genre :Action
Description :["Les destins de deux soldats romains et de leurs familles alors que la République Romaine est en train de s'effondrer en laissant peu à peu la place à un Empire."]
Titre :Code 37, affaires de moeurs
score de similarité :10%
Genre :Police
Description :['Hannah Maes est nommée capitaine au sein de la brigade des mœurs de Gand où elle dirige avec tempérament et finesse sa nouvelle équipe exclusivement masculine : Charles adepte des méthodes traditionnelles de la vieille école, Bob le macho et le jeune Kevin, spécialiste des nouvelles technologies. Malgré leur scepticisme, les enquêt

Recherche par description en utilisant un algo supervisé

Avec TF (CountVectorizer)

In [ ]:
#Générer la matrice des poids des descriptions pour appliquer les algos
tf_vectorizer = CountVectorizer(binary=False)
tf_desc = tf_vectorizer.fit_transform(desc_clean)


In [ ]:
#Générer les deux échantillons d'entrainement et de test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(tf_desc, genre, train_size = 0.75, random_state=40)

In [ ]:
#Appliquer LogisticRegression sur la matrice de poids en modifiant le parametre de régularisation C pour un bon score d'Accuracy
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
for c in [0.01, 0.05, 0.25, 0.5, 1]:
    lr = LogisticRegression(C=c)
    lr.fit(X_train, y_train)
    print ("Accuracy for C=%s: %s" % (c, accuracy_score(y_test, lr.predict(X_test))))

Accuracy for C=0.01: 0.34134615384615385
Accuracy for C=0.05: 0.3269230769230769
Accuracy for C=0.25: 0.33653846153846156
Accuracy for C=0.5: 0.32211538461538464
Accuracy for C=1: 0.32211538461538464


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [ ]:
#Appliquer SVC sur la matrice de poids en modifiant le parametre de régularisation C pour un bon score d'Accuracy
from sklearn.svm import LinearSVC
for c in [0.01, 0.05, 0.25, 0.5, 1]:
    svm = LinearSVC(C=c)
    svm.fit(X_train, y_train)
    print ("Accuracy for C=%s: %s" % (c, accuracy_score(y_test, svm.predict(X_test))))


Accuracy for C=0.01: 0.35336538461538464
Accuracy for C=0.05: 0.35096153846153844


/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


Accuracy for C=0.25: 0.31971153846153844
Accuracy for C=0.5: 0.3173076923076923
Accuracy for C=1: 0.31009615384615385


In [ ]:
#Appliquer l'arbre de décision sur la matrice de poids 
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
tr = DecisionTreeClassifier(criterion = "gini", random_state = 100,max_depth=3, min_samples_leaf=5)
tr.fit(X_train, y_train)
print ("Accuracy : %s" % accuracy_score(y_test, tr.predict(X_test)))

Accuracy : 0.23076923076923078


In [ ]:
#Appliquer KNN sur la matrice de poids en modifiant le parametre K pour un bon score d'Accuracy
from sklearn.neighbors import KNeighborsClassifier
k_epoch=range(1,400,10)
for k in k_epoch:
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train, y_train)
    print ("Accuracy for K=%s: %s" % (k, accuracy_score(y_test, knn.predict(X_test))))

Accuracy for K=1: 0.1346153846153846
Accuracy for K=11: 0.15384615384615385
Accuracy for K=21: 0.17067307692307693
Accuracy for K=31: 0.17307692307692307
Accuracy for K=41: 0.16346153846153846
Accuracy for K=51: 0.18028846153846154
Accuracy for K=61: 0.20432692307692307
Accuracy for K=71: 0.22596153846153846
Accuracy for K=81: 0.2283653846153846
Accuracy for K=91: 0.22596153846153846
Accuracy for K=101: 0.23076923076923078
Accuracy for K=111: 0.2403846153846154
Accuracy for K=121: 0.24519230769230768
Accuracy for K=131: 0.24278846153846154
Accuracy for K=141: 0.23798076923076922
Accuracy for K=151: 0.23557692307692307
Accuracy for K=161: 0.23557692307692307
Accuracy for K=171: 0.23557692307692307
Accuracy for K=181: 0.23076923076923078
Accuracy for K=191: 0.23317307692307693
Accuracy for K=201: 0.23557692307692307
Accuracy for K=211: 0.23557692307692307
Accuracy for K=221: 0.23557692307692307
Accuracy for K=231: 0.23557692307692307
Accuracy for K=241: 0.23076923076923078
Accuracy for K

In [ ]:
#quantifier les label pour appliquer naive_bayes
label=[]
for i in genre:
  if i == "Action" :
    label.append(1)
  if i == "Aventure" :
    label.append(2)
  if i == "Comedie" :
    label.append(3)
  if i == "Drame" :
    label.append(4)
  if i == "Medical" :
    label.append(5)  
  if i == "Police" :
    label.append(6)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(tf_desc, label, train_size = 0.75, random_state=40)

In [ ]:
#Appliquer naive_bayes sur la matrice de poids 
from sklearn.naive_bayes import MultinomialNB
naive_bayes = MultinomialNB()
naive_bayes.fit(X_train, y_train)
print ("Accuracy : %s" % accuracy_score(y_test, naive_bayes.predict(X_test)))

Accuracy : 0.3317307692307692


Avec TfIdf (TfidfVectorizer)

In [ ]:
#Générer la matrice des poids des descriptions pour appliquer les algos
tfidf_vectorizer = TfidfVectorizer(binary=False)
tfidf_desc = tfidf_vectorizer.fit_transform(desc_clean)

In [ ]:
#Générer les deux échantillons d'entrainement et de test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(tfidf_desc, genre, train_size = 0.75, random_state=40)

In [ ]:
#Appliquer LogisticRegression sur la matrice de poids en modifiant le parametre de régularisation C pour un bon score d'Accuracy
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
for c in [0.01, 0.05, 0.25, 0.5, 1]:
    lr = LogisticRegression(C=c)
    lr.fit(X_train, y_train)
    print ("Accuracy for C=%s: %s" % (c, accuracy_score(y_test, lr.predict(X_test))))

Accuracy for C=0.01: 0.18028846153846154
Accuracy for C=0.05: 0.2764423076923077
Accuracy for C=0.25: 0.3389423076923077
Accuracy for C=0.5: 0.3581730769230769
Accuracy for C=1: 0.3581730769230769


In [ ]:
#Appliquer SVC sur la matrice de poids en modifiant le parametre de régularisation C pour un bon score d'Accuracy
from sklearn.svm import LinearSVC
for c in [0.01, 0.05, 0.25, 0.5, 1]:
    svm = LinearSVC(C=c)
    svm.fit(X_train, y_train)
    print ("Accuracy for C=%s: %s" % (c, accuracy_score(y_test, svm.predict(X_test))))


Accuracy for C=0.01: 0.29086538461538464
Accuracy for C=0.05: 0.38221153846153844
Accuracy for C=0.25: 0.3701923076923077
Accuracy for C=0.5: 0.36538461538461536
Accuracy for C=1: 0.36538461538461536


In [ ]:
#Appliquer l'arbre de décision sur la matrice de poids
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
tr = DecisionTreeClassifier(criterion = "gini", random_state = 100,max_depth=3, min_samples_leaf=5)
tr.fit(X_train, y_train)
print ("Accuracy : %s" % accuracy_score(y_test, tr.predict(X_test)))

Accuracy : 0.2283653846153846


In [ ]:
#Appliquer KNN sur la matrice de poids en modifiant le parametre K pour un bon score d'Accuracy
from sklearn.neighbors import KNeighborsClassifier
k_epoch=range(1,400,10)
for k in k_epoch:
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train, y_train)
    print ("Accuracy for K=%s: %s" % (k, accuracy_score(y_test, knn.predict(X_test))))

Accuracy for K=1: 0.0985576923076923
Accuracy for K=11: 0.16346153846153846
Accuracy for K=21: 0.15384615384615385
Accuracy for K=31: 0.16826923076923078
Accuracy for K=41: 0.15865384615384615
Accuracy for K=51: 0.24519230769230768
Accuracy for K=61: 0.2668269230769231
Accuracy for K=71: 0.25961538461538464
Accuracy for K=81: 0.27884615384615385
Accuracy for K=91: 0.3004807692307692
Accuracy for K=101: 0.33653846153846156
Accuracy for K=111: 0.35336538461538464
Accuracy for K=121: 0.3629807692307692
Accuracy for K=131: 0.3894230769230769
Accuracy for K=141: 0.40865384615384615
Accuracy for K=151: 0.4230769230769231
Accuracy for K=161: 0.4326923076923077
Accuracy for K=171: 0.44711538461538464
Accuracy for K=181: 0.43990384615384615
Accuracy for K=191: 0.4495192307692308
Accuracy for K=201: 0.46153846153846156
Accuracy for K=211: 0.46875
Accuracy for K=221: 0.4495192307692308
Accuracy for K=231: 0.4495192307692308
Accuracy for K=241: 0.4375
Accuracy for K=251: 0.45913461538461536
Accura

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(tf_desc, label, train_size = 0.75, random_state=40)

In [ ]:
#Appliquer naive_bayes sur la matrice de poids
from sklearn.naive_bayes import MultinomialNB
naive_bayes = MultinomialNB()
naive_bayes.fit(X_train, y_train)
print ("Accuracy : %s" % accuracy_score(y_test, naive_bayes.predict(X_test)))

Accuracy : 0.3317307692307692


Recherche par genre

In [ ]:
#fct qui génére un score(moyenne des avis) d'un film donné en se basant sur l'analyse de sentiment des avis
def scorer_film(tit):
  res=[]
  sc=0
  from textblob import TextBlob
  from textblob import Blobber
  from textblob_fr import PatternTagger, PatternAnalyzer
  tb = Blobber(pos_tagger=PatternTagger(), analyzer=PatternAnalyzer())
  for i in range(0,len(titreR)):
       if titreR[i] == tit:
           text = tb(u""+avis[i])
           res.append(text.sentiment[0])
       elif len(res) != 0.0:
           sc = sum(res)/len(res)
  return round(sc,2)

#fct qui retourne les film d'un genre donné ordonné par un score d'analyse de sentiment
def scorer_par_genre(g):
  import pandas as pd
  film=[]
  score=[]
  for i in range(0,len(genre)):
    if (genre[i] == g) and (scorer_film(titre[i]) !=0.0) :
         film.append(titre[i])
         score.append(int(scorer_film(titre[i])*100))
  d={"Titre":film,"Score":score}
  df=pd.DataFrame(d)
  df=df.dropna(axis=0)
  df=df.sort_values(by=['Score'], ascending=False)
  df=df.iloc[0:10,]
  df=df.reset_index(drop=True)
  return df

In [ ]:
scorer_par_genre("Action")


,Titre,Score
0,Halo: The Fall Of Reach,70
1,Marvel's Spider-Man,62
2,Ken : Fist of the Blue Sky Regenesis,61
3,Chicago Fire,40
4,Le Rebelle,39
5,24 : La conspiration,36
6,Simon et Simon,36
7,Hunter X Hunter (2011),34
8,Kengan Ashura,34
9,Dragons : par-delà les rives,34


In [ ]:
#Le film le plus similaire a une description donnée si la requete entrante est une discription ou une partie de cette dérnière
def sim_desc(query):
  query=preprocessing(query)
  import numpy as np
  tfidf_vectorizer = TfidfVectorizer()
  tfidf_desc = tfidf_vectorizer.fit_transform(desc_clean)
  q = tfidf_vectorizer.transform([query])
  cs = cosine_similarity(q, tfidf_desc)
  ind=np.argmax(cs[0])
  print("Titre  : "+titre[ind])
  print("Description  : "+desc[ind])




In [ ]:
sim_desc("guerre problème")

Titre  : L'Enfer du devoir
Description  : ["Au coeur de la guerre du Vietnam avec une unité d'infanterie de l'armée américaine..."]


In [ ]:
#Les bon films ordonés par analyse de sentiment des avis
def positive_film():
  res=[]
  ress=[]
  for i in titre:
    res.append(i)
    ress.append(int(scorer_film(i)*100))
  d={"Titre":res,"Score":ress}
  df=pd.DataFrame(d)
  df=df.dropna(axis=0)
  df=df.sort_values(by=['Score'], ascending=False)
  df=df.iloc[0:10,]
  df=df.reset_index(drop=True)
  df=df.drop_duplicates()
  return df


In [ ]:
#Les mauvais films ordonés par analyse de sentiment des avis
def negative_film():
  res=[]
  ress=[]
  for i in titre:
    res.append(i)
    ress.append(int(scorer_film(i)*100))
  d={"Titre":res,"Score":ress}
  df=pd.DataFrame(d)
  df=df.dropna(axis=0)
  df=df.sort_values(by=['Score'])
  df=df.iloc[0:10,]
  df=df.reset_index(drop=True)
  df=df.drop_duplicates()
  return df

Recherche générale

In [ ]:
#fct qui retourne:
######Le titre si la requete entrante est un nom d'un film
######Les différentes films d'un genre donné grouper par analyse des avis si  la requete entrante est un genre
######Le film le plus similaire a une description donnée si la requete entrante est une discription ou une partie de cette dérnière
######Les bon films ordonés par analyse de sentiment des avis si la requete entrante est un avis positif 
######Les mauvais films ordonés par analyse de sentiment des avis si la requete entrante est un avis négatif
def recherche_generale(requete):
  from textblob import TextBlob
  from textblob import Blobber
  from textblob_fr import PatternTagger, PatternAnalyzer
  tb = Blobber(pos_tagger=PatternTagger(), analyzer=PatternAnalyzer())
  text = tb(u""+requete)
  res=text.sentiment
  if res[1] == 0.0 :
    for i in titre:
      if i == requete :
#        print("recherche par titre")
        return recherche_par_titre_AS(requete)
    for i in list(set(genre)):
      if i == requete :
#        print("recherche par genre")
        return  scorer_par_genre(requete) 
  elif (res[0] < 0.3 and res[0] > 0) or (res[0] > -0.3 and res[0] < 0) :
#    print("recherche par description")
    return sim_desc(requete)
  elif res[0] > 0.3 :
#    print("Positivité")
    return positive_film()
  elif (res[0] < int(-1*0.3)) :
#    print("Négativité")
    return negative_film()

 


In [ ]:
desc[0]

'["Dick Grayson, alias Robin,\\xa0sort de l’ombre pour devenir le chef de file d’un groupe de nouveaux héros constitué notamment de Starfire, Raven ou encore Hawk et Dove. Peu à peu, il\\xa0s\'affranchit de Batman pour devenir Nightwing...."]'

In [ ]:
recherche_generale("bon film")

,Titre,Score
0,October Faction,72
1,El Síndrome de Ulises,70
2,Halo: The Fall Of Reach,70
3,Carmen Sandiego,65
4,Marvel's Spider-Man,62
5,Ken : Fist of the Blue Sky Regenesis,61
6,Le Pacte,51
7,Les Aventures de Sinbad,50
8,Alliances & trahisons,47
9,Pagan Peak,47
